In [49]:
from IPython.display import HTML

text = str('''
    <script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [50]:
HTML(text)

# Sensitivity Analysis

## Explanation of Notebook

This notebook started out addressing a dataset related to my internship with Earth Economics. I was asked to read through a research paper (PDF shown below), and use the sensitivity analysis method shown on the data provided to me related to the organizations project. I am not able to show the data from Earth Economics, but I can show the process applied to the data in the research paper shown below.

To start off, I will show the data and tables in the research paper, along with how each quantity is calculated.

In [48]:
file = "https://agrilife.org/kreuter/files/2013/01/Change-in-ecosystem-service-values-in-SanAntonio-areaTexas_6.pdf"
HTML("<iframe width = 90% height = 700 src = " + file + "></iframe")

## Explanation of Data

So the goal of this research paper, and what we were trying to do at Earth Economics was something called Ecosystem Service Valuation (ESV). In the paper above, there are two inputs to this, the number of acres of a given land area and type, and the ecosystem service benefit coefficient corresponding to that land type (in dollars per acre per year). There will be many different land types in a given study area, and each will have a different benefit estimate. 

The total ecosystem service value of a given study area will be in units of dollars per year. In the paper above there are six land cover types, and the acreage of each was measured through satellite images, and some GIS software. To get the total ESV, each land type benefit coefficient amount (VC) is multiplied by its corresponding acreage, then these are all added together.

Put into a formula: $$ESV = \sum_{i=1}^{n} VC_i \cdot A_i \\ $$
$$VC_i = \text{benefit coefficient amount for land type i} \\ $$
$$A_i = \text{number of Acres for land type i} \\ $$
$$\text{The sum is over the n land types} \\ $$

In [15]:
def html_table(head, row):
    cols = len(head)
    head_text = '<tr>'
    for header in head:
        head_text += '<th>' + str(header) + '</th>'
    head_text += '</tr>'
    row_text = ''
    counter = 0
    for element in row:
        if counter == 0:
            row_text += '<tr>'
        counter += 1
        row_text += '<td>' + str(element) + '</td>'
        if counter == cols:
            row_text += '</tr>'
            counter = 0
    html_code = str(head_text) + str(row_text)
    return html_code
def html_tables(tables, cols):
    counter = 0
    position = ['left','center','right']
    text = '<html>'
    for table in tables:
        text += '<table align=' + position[counter] + '>' + table + '</table>'
        counter += 1
        if counter == cols:
            counter = 0
    text += '</html>'
    
    return HTML(text)

In [16]:
tables = []
head = ['Land Cover<br>categories',
            'Equivalent biome',
            'Ecosystem service<br />coefficient<br>($/ha per year)']
row = ['Rangeland','Grass/rangelands','232',
        'Woodland','Temperate/boreal forest','302',
        'Bare Soil','Cropland','92',
        'Residential','Urban','0',
        'Commercial','Urban','0',
        'Transportation','Urban','0']

head1 = ['Land-Use Category', 
            'Total Area (ha)<br>____<br>1976',
            '<br>____<br>1985',
            '<br>____<br>1991']
row1 = ['Rangeland', '80,497','59,126','27,896',
        'Woodland','8,886', '25,336','44,654',
        'Bare Soil','6,353','13,514','13,047',
        'Residential','11,499','10,087','16,655',
        'Commercial','6,116','10,457','15,362',
        'Transportation','25,748','23,060','23,857']

head2 = ['Land-Use Category', 
            'ESV<br>____<br>1976',
            '(US$×10^6<br>____<br>1985',
            'per  year)<br>____<br>1991']
row2 = ['Rangeland', '18.68','13.72','6.47',
        'Woodland','2.68', '7.65','13.49',
        'Bare Soil','0.58','1.24','1.20',
        'Urban Categories','0.00','0.00','0.00',
        'Total','21.94','22.61','21.16']

head3 = ['Change  in  valuation  coefficient  (VC)', 
            'ESV<br>____<br>1976',
            '<br>____<br>1991',
            'CS<br>____<br>1976',
            '<br>____<br>1991']
row3 = ['Rangeland  VC+50%', '31.28','24.39','0.85','0.31',
        'Rangeland  VC-50%','12.61', '17.92','0.85','0.31',
        'Woodland  VC+50%','23.28','27.90','0.12','0.64',
        'Woodland  VC-50%','20.60','14.41','0.12','0.64',
        'Woodland  VC=Rangeland  VC','21.32','18.03','0.12','0.64',
        'Bare  soil  VC+50%','22.24','21.76','0.03','0.06',
        'Bare  soil  VC-50%','21.65','20.56','0.03','0.06']

heads = [head, head1, head2,head3]
rows = [row, row1, row2,row3]

for i in range(len(heads)):
    tables.append(html_table(heads[i], rows[i]))

html_tables(tables, 2)

### The elasticity equation from Urs' paper, and its independence from the adjustment percentage

**The equation in Urs' paper for the coefficient of sensitivity (CS) of the adjusted ESV with respect to the adjusted VC for a given adjustment x is:**

$$CS_x = \frac{\frac{ESV_x - ESV}{ESV}}{\frac{VC_{kx} - VC_k}{VC_k}} \\ $$
$$\text{For any given adjustment percentage x and land type k, and using the ESV definition above:} \\ $$
$$ESV_x = ESV - (A_k\cdot VC_k) + (A_k\cdot (x\cdot VC_k)) \\ $$
$$\text{So:} \ ESV_x = ESV + (x-1)\cdot (A_k\cdot VC_k) \\ $$
$$\text{Similarly, the adjustment to} \ VC_k, \ VC_{kx} = x\cdot VC_k \\ $$
$$\text{If we subsitute the values above for} \ ESV, \ ESV_x, \ VC_k, \ VC_{kx}, \text{the numerator will be:} \\ $$ $$\frac{(ESV + (x-1)\cdot (A_k\cdot VC_k)) - ESV}{ESV} \\ $$
$$\text{and canceling out the ESV terms on top:} \quad\ \frac{(x-1)\cdot A_k\cdot VC_k}{ESV} \\ $$
$$\text{For the denominator, we have} \ \frac{x\cdot VC_k - VC_k}{VC_k} = \frac{(x-1)\cdot VC_k}{VC_k} = (x-1) \\ $$ 
$$\text{Combining the numerator and the denominator:} \ CS_x = \frac{(x-1)\cdot A_k\cdot VC_k}{ESV\cdot (x-1)} \\ $$
$$\text{Canceling out the (x-1) terms, we are left with:} \ CS_x = \frac{A_k\cdot VC_k}{ESV} \\ $$
$$\text{So, since the coefficient is independent of x:} \ CS = \frac{A_k\cdot VC_k}{ESV}$$

### The Midpoint Elasticity formula more generally used in Economics

$$\text{Define the Midpoint Elasticity formula as: } MP_x = \frac{\frac{ESV_x - ESV}{avg(ESV_x, ESV)}}{\frac{VC_{kx} - VC_k}{avg(VC_{kx}, VC_k)}} \\ $$
$$\text{Simplifying the fraction, we get: } MP_x = \frac{(ESV_x - ESV)\cdot avg(VC_{kx}, VC_x)}{(VC_{kx} - VC_x)\cdot avg(ESV_x, ESV)} \\ $$
$$ESV_x - ESV = (x-1)\cdot A_k\cdot VC_k \\ $$
$$avg(ESV_x, ESV) = \frac{ESV + (x-1)\cdot A_k\cdot VC_k + ESV}{2} = \frac{2\cdot ESV + (x-1)\cdot A_k\cdot VC_k}{2} \\ $$
$$VC_{kx} - VC_x = x\cdot VC_k - VC_k = (x-1)\cdot VC_k \\ $$
$$avg(VC_{kx}, VC_x) = \frac{x\cdot VC_k + VC_k}{2} = \frac{(x+1)\cdot VC_k}{2} \\ $$
$$(ESV_x - ESV)\cdot avg(VC_{kx}, VC_x) = \frac{((x-1)\cdot A_k\cdot VC_k)\cdot ((x+1)\cdot VC_k)}{2} \\ $$
$$(VC_{kx} - VC_x)\cdot avg(ESV_x, ESV) = \frac{((x-1)\cdot VC_k)\cdot (2\cdot ESV + (x-1)\cdot A_k\cdot VC_k)}{2} \\ $$
$$\text{Putting this together into one fraction, we can cancel out two of the (x-1)'s, two of the } VC_k\text{, and the two 2's} \\ $$
$$MP_x = \frac{(x+1)\cdot A_k\cdot VC_k}{2\cdot ESV + (x-1)\cdot A_k\cdot VC_k}$$

### Setting up the data for elasticity calculations and analysis

In [18]:
#import the necessary libraries for processing the data
import seaborn as sns #Statistical graphing library
import matplotlib.pyplot as plt #Basic graphing library
import random #Library for random number generation
from decimal import *
getcontext().prec = 4

#Setting so that the graphs appear within this notebook
%matplotlib inline

**I grouped the data by Land Cover Type, Land Use, Ecosystem Service, Value Type, County, and MLRA.**

### Define Functions to calculate and graph the elasticity

In [19]:
def elasticity(attribute): #Function to calculate elasticity as per Urs' equation
    
    #Define dictionaries to store the values
    low_elasticity, high_elasticity = {}, {}
    attributes = list(data[attribute].unique())
    
    low_data = data[data['Value Type'] == 'Low']
    high_data = data[data['Value Type'] == 'High']
    
    for unit in attributes:
        low_unit_data = data[(data[attribute] == unit)&
                             (data['Value Type'] == 'Low')]
        high_unit_data = data[(data[attribute] == unit)&
                              (data['Value Type'] == 'High')]
        
        low_elasticity[str(unit)], high_elasticity[str(unit)] = 0, 0

        low_unit_ESV = np.sum(low_unit_data['#Acres']*
                              low_unit_data['$/acre/year']*
                              low_unit_data['Health_Index'])
        low_ESV = np.sum(low_data['#Acres']*
                         low_data['$/acre/year']*
                         low_data['Health_Index'])
        high_unit_ESV = np.sum(high_unit_data['#Acres']*
                               high_unit_data['$/acre/year']*
                               high_unit_data['Health_Index'])
        high_ESV = np.sum(high_data['#Acres']*
                          high_data['$/acre/year']*
                          high_data['Health_Index'])
                
        low_elasticity[str(unit)] = low_unit_ESV/low_ESV
        high_elasticity[str(unit)] = high_unit_ESV/high_ESV
    
    return low_elasticity, high_elasticity

In [20]:
def midpoint_elasticity(attribute, adjustments):
    
    low_elasticity, high_elasticity = {}, {}
    attributes = list(data[attribute].unique())
    low_data = data[data['Value Type'] == 'Low']
    high_data = data[data['Value Type'] == 'High']
    x = 0
    
    for unit in attributes:
        low_unit_data = data[(data[attribute] == unit)&
                             (data['Value Type'] == 'Low')]
        high_unit_data = data[(data[attribute] == unit)&
                              (data['Value Type'] == 'High')]
        
        low_elasticity[str(unit)], high_elasticity[str(unit)] = [], []
        
        for pct in adjustments:
            x = (100 + pct)/100
            
            low_unit_ESV = np.sum(low_unit_data['#Acres']*
                                  low_unit_data['$/acre/year']*
                                  low_unit_data['Health_Index'])
            low_ESV = np.sum(low_data['#Acres']*
                             low_data['$/acre/year']*
                             low_data['Health_Index'])
            high_unit_ESV = np.sum(high_unit_data['#Acres']*
                                   high_unit_data['$/acre/year']*
                                   high_unit_data['Health_Index'])
            high_ESV = np.sum(high_data['#Acres']*
                              high_data['$/acre/year']*
                              high_data['Health_Index'])
            
            MP_low = ((x+1)*(low_unit_ESV))/((2*low_ESV) + (x-1)*(low_unit_ESV))
            MP_high = ((x+1)*(high_unit_ESV))/((2*high_ESV) + (x-1)*(high_unit_ESV))
            low_elasticity[str(unit)].append(MP_low)
            high_elasticity[str(unit)].append(MP_high)
                                              
    return low_elasticity, high_elasticity

In [21]:
def graph_by_attribute(low_elasticity, high_elasticity, attribute, adjustments, midpoint = True):
        
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12,30), 
                                   sharey = True, sharex = False)
    low_plot = []
    high_plot = []
    attributes = sorted(list(data[attribute].unique()), reverse = True)
    
    for unit in attributes:
        ax1.plot(adjustments, low_elasticity[str(unit)], label = str(unit))
        ax2.plot(adjustments, high_elasticity[str(unit)], label = str(unit))

    ax1.set_xlabel('Percent Adjustment'); ax2.set_xlabel('Percent Adjustment')
    if midpoint:
        ax1.set_title('Low Estimate: Midpoint Formula'); ax1.set_ylabel('Elasticity')
        ax2.set_title('High Estimate: Midpoint Formula'); ax2.set_ylabel('Elasticity')
    else:
        ax1.set_title("Low Estimate: Urs' Formula"); ax1.set_ylabel('Elasticity')
        ax2.set_title("High Estimate: Urs' Formula"); ax2.set_ylabel('Elasticity')
    ax1.legend(loc = 'upper left', ncol = 3, fancybox = True); ax2.legend(loc = 'upper left', ncol = 3)
    
    return None

In [22]:
def graph_by_estimate(low_elasticity, high_elasticity, attribute, midpoint = True):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,30), 
                                   sharey = True, sharex = True)
    low_plot = []
    high_plot = []
    attributes = sorted(list(data[attribute].unique()), reverse = True)
    
    for unit in attributes:
        low_plot.append(np.mean(low_elasticity[str(unit)]))
        high_plot.append(np.mean(high_elasticity[str(unit)]))

    x_range = list(range(len(attributes)))
    ax1.set_yticks(x_range)
    ax1.set_yticklabels(attributes)
    ax2.set_yticks(x_range)
    ax2.set_yticklabels(attributes)
    
    ax1.barh(bottom = x_range, width = low_plot, label = 'Low Estimate')
    ax2.barh(bottom = x_range, width = high_plot, label = 'High Estimate')
    if midpoint:
        ax1.set_title("Low Estimate: Midpoint Formula"); ax1.set_xlabel('Elasticity')
        ax2.set_title("High Estimate: Midpoint Formula"); ax2.set_xlabel('Elasticity')
    else:
        ax1.set_title("Low Estimate: Urs' Formula"); ax1.set_xlabel('Elasticity')
        ax2.set_title("High Estimate: Urs' Formula"); ax2.set_xlabel('Elasticity')
    
    return None

### Set up Simulation Functions and Methodology in the Literature

**Most of the methods I found in the ecosystem literature and in the academic literature in general involve running simulations by pulling random values for each input and assessing the effects on the output.**

**Literature examples:**

*From screening to quantitative sensitivity analysis. A unified approach*

*Sensitivity analysis ecosystem service valuation Sanchez-Canales et al 2012*

*Sensitivity analysis: how to detect important factors in large models*


**In general, these types of methods are used for complex models where the partial derivative with respect to the inputs are to laborous/impossible to calculate. Many of these methods (such as the Morris/Elementary Effects method) are just aggregates of the estimates of the derivates**

**For this data and model, it is possible to calculate the partial derivatives with respect to each of the inputs ($/acre/year, Health Score, and #Acres)**

$$\text{As above: The total ecosystem service value is:} \ ESV = \sum_{i=1}^{n} A_i\cdot VC_i \\ $$
$$A_i =\text{Acres} \quad\ VC_i =\frac{\text{Dollars}}{\text{Acre}\cdot \text{Year}} n =\text{Number of Land Types} \\ $$
$$\text{The valuation coefficient for the kth attribute in the category is:} \ VC_k \\ $$
$$\text{The partial derivative of the total ESV with respect to the kth valuation coefficients } VC_k \ \text{is: } \\ $$
$$\frac{\partial}{\partial VC_k}(\sum_{i=1}^{n} A_i\cdot VC_i) = \sum_{i=1}^{n} \frac{\partial}{\partial VC_k}(A_i\cdot VC_i) \\ $$

In [23]:
def get_test_data():
    test_data = data
    test_data['$/acre/year'] = test_data.apply(lambda x: 
                                               np.random.uniform(.75*x['Low $/acre/year'], 
                                                                 1.25*x['High $/acre/year']), axis = 1)
    return test_data

def ESV_by_attribute(test_data, attribute):
    ESV_dict = {}
    attributes = list(test_data[attribute].unique())
    total = np.sum(test_data['#Acres']*test_data['Health_Index']*test_data['$/acre/year'])
    ESV_dict['Total'] = total
    
    for element in attributes:
        test_data_trim = test_data[test_data[attribute] == element]
        total_trim = np.sum(test_data_trim['#Acres']*
                            test_data_trim['Health_Index']*
                            test_data_trim['$/acre/year'])
        ESV_dict[str(element)] = total_trim
        
    return ESV_dict

def VC_by_attribute(test_data, attribute):
    VC_dict = {}
    attributes = list(test_data[attribute].unique())
    total = np.sum(test_data['$/acre/year'])
    VC_dict['Total'] = total
    
    for element in attributes:
        test_data_trim = test_data[test_data[attribute] == element]
        total_trim = np.sum(test_data_trim['$/acre/year'])
        VC_dict[str(element)] = total_trim
        
    return VC_dict

def monte_carlo(NumTrials, attribute):
    
    attributes = list(data[attribute].unique())
    ESV_vals, VC_vals = {}, {}
    ESV_vals['Total_ESV'], VC_vals['Total_VC'] = [], []
    
    for element in attributes:
        VC_vals[str(element) + '_VC'], ESV_vals[str(element) + '_ESV'] = [], []
        
    for trial in range(NumTrials):
        
        test_data = get_test_data()
        ESV_dict = ESV_by_attribute(test_data, attribute)
        VC_dict = VC_by_attribute(test_data, attribute)
        ESV_vals['Total_ESV'].append(ESV_dict['Total'])
        VC_vals['Total_VC'].append(VC_dict['Total'])
        for element in attributes:
            VC_vals[str(element) + '_VC'].append(VC_dict[element])
            ESV_vals[str(element) + '_ESV'].append(ESV_dict[element])
    
    plot_data = pd.concat([pd.DataFrame.from_dict(ESV_vals),
                           pd.DataFrame.from_dict(VC_vals)], axis = 1)
    
    return plot_data

In [24]:
def jacobian(attribute, variable):
    attributes = list(data[attribute].unique())
    cols = ['#Acres', 'Low $/acre/year', 'High $/acre/year', 'Health_Index']
    if variable == '$/acre/year':
        variables = ['Low $/acre/year', 'High $/acre/year']
    else:
        variables = variable
    cols.remove(variables)
    print(cols)